# NLP con Deep Learning: Transformers

- [Curso simple Coursera](https://www.coursera.org/)

- [Curso Avanzado Stanford](https://web.stanford.edu/class/cs224n/index.html#schedule)

- [Implementar Transformers para Clasificacion](https://huggingface.co/transformers/v3.2.0/custom_datasets.html)

In [2]:
import torch
import numpy as np
import pandas as pd

In [40]:
#device = "mps" # para mac M1 en adelante
device = "cpu"
#device = "cuda:0" if torch.cuda.is_available() else "cpu" #para gpu
print(device)

cpu


In [4]:
df_train = pd.read_parquet('https://github.com/amiune/amiune.github.io/raw/master/movie-genre-prediction/train.parquet', engine='pyarrow')

In [5]:
df_train.head()

,id,movie_name,synopsis,genre
0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action


In [6]:
df_train.shape

(54000, 4)

In [7]:
df_train.genre = df_train.genre.astype('category')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          54000 non-null  int64   
 1   movie_name  54000 non-null  object  
 2   synopsis    54000 non-null  object  
 3   genre       54000 non-null  category
dtypes: category(1), int64(1), object(2)
memory usage: 1.3+ MB


In [8]:
df_train.genre.value_counts()

action       5400
adventure    5400
crime        5400
family       5400
fantasy      5400
horror       5400
mystery      5400
romance      5400
scifi        5400
thriller     5400
Name: genre, dtype: int64

In [9]:
id2label = dict(zip(df_train.genre.cat.codes, df_train.genre))
print(id2label)
label2id = dict(zip(df_train.genre, df_train.genre.cat.codes))
print(label2id)

{4: 'fantasy', 5: 'horror', 3: 'family', 8: 'scifi', 0: 'action', 2: 'crime', 1: 'adventure', 6: 'mystery', 7: 'romance', 9: 'thriller'}
{'fantasy': 4, 'horror': 5, 'family': 3, 'scifi': 8, 'action': 0, 'crime': 2, 'adventure': 1, 'mystery': 6, 'romance': 7, 'thriller': 9}


In [10]:
num_classes = len(df_train.genre.value_counts())
print(num_classes)

10


In [11]:
df_train.iloc[0,:]["synopsis"]

'A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.'

In [12]:
def read_columns(df, text_column, label_column):
    texts = df[text_column].tolist()
    labels = df[label_column].cat.codes.tolist()
    return texts, labels

In [13]:
train_texts, train_labels = read_columns(df_train.iloc[0:10,:], "synopsis","genre")

In [14]:
print(len(train_texts),len(train_labels))

10 10


In [15]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2)

In [16]:
print(len(train_texts),len(train_labels))

8 8


# HF Transformers

In [17]:
!pip install transformers[torch] --quiet
#!pip install accelerate -U --quiet

zsh:1: no matches found: transformers[torch]


In [41]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [42]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [43]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [44]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [45]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #item es un diccionario que contiene tres keys: 'input_ids', 'attention_mask' y 'labels'
        #cada key contiene el tensor correspodiente al indice idx
        #item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item = {}
        item['input_ids'] = torch.tensor(self.encodings['input_ids'][idx])
        item['attention_mask'] = torch.tensor(self.encodings['attention_mask'][idx])
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, train_labels)
val_dataset = MyDataset(val_encodings, val_labels)

In [46]:
print(len(train_dataset),len(val_dataset))

8 2


In [47]:
train_dataset[0]

{'input_ids': tensor([  101,  4965,  2154,  1011,  2176,  2273,  4235,  1011,  4237,  1999,
          2166,  1011,  2011,  2305,  6281,  2142,  1999,  2028,  2954, 18834,
          2075,  1996,  2543,  1997,  2037,  8111,  2114,  1996,  6283,  6728,
         20110,  5668,  1012,   102,     0,     0,     0,     0,     0,     0,
             0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(0)}

In [48]:
!pip install evaluate --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    dataloader_pin_memory=False,     # remove if possible for faster training
    evaluation_strategy = "epoch",
    output_dir="./results"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.

In [50]:
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /Users/hernanamiune/opt/anaconda3/lib/python3.9/site-packages/transformers/trainer.py:1645 in    │
│ train                                                                                            │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /Users/hernanamiune/opt/anaconda3/lib/python3.9/site-packages/transformers/trainer.py:1938 in    │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /Users/hernanamiune/opt/anaconda3/lib/python3.9/site-packages/transformers/trainer.py:2759 in    │
│ training_step                                                                                    │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /Users/hernanamiune/opt/anaconda3/lib/python3.9/site-packag

In [ ]:
trainer.evaluate()

In [ ]:
#from transformers import DistilBertForSequenceClassification
#model = DistilBertForSequenceClassification.from_pretrained("amiune/bert-clasificacion-peliculas", num_labels=num_classes).to(device)

In [ ]:
encoding = tokenizer("This is a movie about a couple in love", truncation=True, padding=True, return_tensors="pt").to(device)
encoding

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3185, 2055, 1037, 3232, 1999, 2293,  102]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
outputs = model(**encoding)

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 10])

In [ ]:
# calcular probabilidades
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
# transformar predicciones a nombres
predicted_labels = [(probs[idx].item(), id2label[idx]) for idx, label in enumerate(probs)]
list(reversed(sorted(predicted_labels)))

[(0.5358123183250427, 'crime'),
 (0.5237465500831604, 'horror'),
 (0.5168167948722839, 'fantasy'),
 (0.5101510286331177, 'action'),
 (0.5030176639556885, 'scifi'),
 (0.5011835098266602, 'family'),
 (0.49890342354774475, 'mystery'),
 (0.48189038038253784, 'romance'),
 (0.4747811555862427, 'thriller'),
 (0.468729168176651, 'adventure')]

In [ ]:
max(predicted_labels)[1]

'crime'

In [ ]:
len(val_texts)

16200

## Como predecir nuevos valores en batch

In [ ]:
test_encoding = tokenizer(["This is a movie about a couple in love",
                      "This is a very scary movie"],
                     truncation=True, padding=True,  return_tensors="pt").to(device)
outputs = model(**test_encoding)
logits = outputs.logits
logits.shape

torch.Size([2, 10])

In [ ]:
# calcular probabilidades
for i in range(logits.shape[0]):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(logits[i].squeeze().cpu())
  # transformar predicciones a nombres
  predicted_labels = [(probs[idx].item(), id2label[idx]) for idx, label in enumerate(probs)]
  print(list(reversed(sorted(predicted_labels))))
  print(max(predicted_labels)[1])

[(0.9866400361061096, 'romance'), (0.8892194032669067, 'family'), (0.5927843451499939, 'fantasy'), (0.3652147054672241, 'thriller'), (0.31700220704078674, 'action'), (0.25062334537506104, 'crime'), (0.1991966962814331, 'mystery'), (0.18163041770458221, 'adventure'), (0.14182519912719727, 'scifi'), (0.0764179527759552, 'horror')]
romance
[(0.9810206890106201, 'horror'), (0.8305793404579163, 'thriller'), (0.5052431225776672, 'fantasy'), (0.48686283826828003, 'mystery'), (0.4394770562648773, 'action'), (0.43945422768592834, 'scifi'), (0.23916655778884888, 'adventure'), (0.18225450813770294, 'crime'), (0.09471964091062546, 'family'), (0.08791545033454895, 'romance')]
horror


In [ ]:
#TODO: add id2label to the model
#from transformers import pipeline
#pipe = pipeline("text-classification", model="amiune/bert-clasificacion-peliculas", device=0)
#pipe("This is a very scary movie")

## Guardar mi modelo localmente en el servidor de Colab y luego descargarlo

In [ ]:
trainer.save_model("./mi_modelo")

In [ ]:
from google.colab import files
files.download('./mi_modelo/pytorch_model.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Loguearse en HuggingFace y guardar mi modelo

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("bert-clasificacion-peliculas")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/amiune/bert-clasificacion-peliculas/commit/240f6725089d6d7fe605a74c1a239fb24842ad6c', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='240f6725089d6d7fe605a74c1a239fb24842ad6c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("bert-clasificacion-peliculas")

CommitInfo(commit_url='https://huggingface.co/amiune/bert-clasificacion-peliculas/commit/3405c8c8bad6f4987c589fad31bc60b6bb48d0ad', commit_message='Upload tokenizer', commit_description='', oid='3405c8c8bad6f4987c589fad31bc60b6bb48d0ad', pr_url=None, pr_revision=None, pr_num=None)

# Cargar el modelo desde HuggingFace y predecir la tabla de Test

In [ ]:
import torch
import numpy as np
import pandas as pd

In [ ]:
#device = "mps" # para mac M1 en adelante
device = "cuda:0" if torch.cuda.is_available() else "cpu" #para gpu
print(device)

cuda:0


In [ ]:
!pip install transformers[torch] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.2 MB/s eta 0:00:00


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("amiune/bert-clasificacion-peliculas").to(device)

In [ ]:
df_test = pd.read_parquet('https://github.com/amiune/amiune.github.io/raw/master/movie-genre-prediction/test.parquet', engine='pyarrow')
df_test.head()

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action


In [ ]:
test_texts = df_test["synopsis"].tolist()

In [ ]:
id2label = {4: 'fantasy', 5: 'horror', 3: 'family', 8: 'scifi', 0: 'action', 2: 'crime', 1: 'adventure', 6: 'mystery', 7: 'romance', 9: 'thriller'}

In [ ]:
test_preds = []
for i in range(len(test_texts)):
  val_encoding = tokenizer(test_texts[i], truncation=True, padding=True,  return_tensors="pt").to(device)
  outputs = model(**val_encoding)
  logits = outputs.logits.cpu().detach().numpy()
  test_preds.append(id2label[np.argmax(logits)])

In [ ]:
df_submission = pd.DataFrame({"id":df_test["id"],"genre":test_preds})

In [ ]:
df_submission.to_csv("submission4.csv", index=False)

In [ ]:
from google.colab import files
files.download('submission4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>